# AIF_sprint8-mercari-kaggle

## 問題の理解

### 目的

メルカリの販売済み出品データを使って、適切な販売価格を提示するアルゴリズムを作成する。

### 問題
回帰
評価関数：RMSLE(価格に1加算し対数を取った値の平均二乗和誤差)



[メルカリのコンペ](https://www.kaggle.com/c/mercari-price-suggestion-challenge)

### 概要

本当の価値がいくらかを知るのは難しい。細かいところがちょっと変わるだけで大きな違いがある。

今のオンライン販売の状況をみるにどんどん規模が大きくなっているし、
服の価格一つとっても季節やブランドによってガラッと変わる。
家電製品は新製品が出てスペックが相対的に落ちると価格に反映される。

日本のユニコーン企業であるメルカリは価格付けについてよく知っている。

メルカリは出品者に売れそうな価格帯を提示したいがなかなかタフだ。
なぜなら元にする商品データは出品者が好き勝手かけるからです。

このコンペでは適切な商品価格を自動的につけるアルゴリズムをつくることが目的です。

提供されるデータは製品のカテゴリ名、ブランド名、品目条件などの詳細を含む、ユーザーが入力した製品のテキストの説明です。

※注意　このデータの性質上このコンペはカーネルだけのコンペです。

ファイルはカーネルを通じてのみ利用可能で、新しいデータに対応してアプローチを修正することはできません。

### データセットの用意

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")

import warnings
warnings.filterwarnings('ignore')

import gc # ガベージコレクションを用意

In [31]:
# ls ../input

In [32]:
path= '../input/'

train_df = pd.read_csv(path + 'train.tsv', sep='\t', encoding='utf-8')
test_df = pd.read_csv(path+'test.tsv', sep='\t', encoding='utf-8')

### EDA


In [33]:
train_df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


商品名、カテゴリーデータ、ブランド名、商品説明欄は文字列となっていて**自然言語処理**が必要となる。

商品説明欄には**No description yet**、つまり説明無しもあるためこれは言葉としてではなく空文字で扱うなど注意が必要な事が分かる。


In [34]:
train_df[train_df['item_description'] == "No description yet"].count()

train_id             82489
name                 82489
item_condition_id    82489
category_name        81867
brand_name           45116
price                82489
shipping             82489
item_description     82489
dtype: int64

実際にNo description yetは8万2489件あります。

In [35]:
train_df.isnull().sum()

train_id                  0
name                      0
item_condition_id         0
category_name          6327
brand_name           632682
price                     0
shipping                  0
item_description          4
dtype: int64

欠損値は商品説明欄、カテゴリー名、ブランド名で見られる。

それぞれの項目は以下の通り。
- train_id リストのID
- name 商品名
- item_condition_id　商品の状態 　
    - 新品、未使用
    - 未使用に近い
    - 目立った傷や汚れなし
    - やや傷や汚れあり
    - 傷や汚れあり
    - 全体的に状態がわるい
- category_name    カテゴリー 
    - 例「本・音楽・ゲーム」->「テレビゲーム」-「家庭用ゲーム本体」
- brand_name   ブランド名
- price  価格　
    - 販売価格は実際に販売された価格　USD単位で今回の目的変数
- shipping  送料どっち負担か？　出荷手数料が売り手によって支払われる場合は1、買い手によって支払われる場合は0
- item_description  商品の説明欄

価格に関する情報が入っている場合は**rm**に変換されている。

In [36]:
train_df.nunique()

train_id             1482535
name                 1225273
item_condition_id          5
category_name           1287
brand_name              4809
price                    828
shipping                   2
item_description     1281426
dtype: int64

今のアプリで確認すると商品の状態は６段階だがUS版は５段階のようです。

In [37]:
train_df[train_df['category_name']=='Kids/Girls 0-24 Mos/Shoes'].groupby(['item_condition_id']).agg('mean')

,train_id,price,shipping
item_condition_id,,,
1,7.340890e+05,18.499444,0.491111
2,7.772671e+05,17.256780,0.472034
3,7.504571e+05,15.844483,0.434534
4,8.384035e+05,13.100775,0.403101
5,1.277316e+06,5.000000,1.000000


In [38]:
train_df[train_df['category_name'].fillna('nan').str.contains('Books')].groupby(['item_condition_id']).agg('mean')

,train_id,price,shipping
item_condition_id,,,
1,715599.378995,17.181996,0.525114
2,735135.971497,16.883581,0.423926
3,745969.350545,15.452381,0.483649
4,809090.604790,13.919162,0.464072
5,936581.625000,13.750000,0.125000


いくつかカテゴリーを絞って見てみると商品状態IDが小さい方が販売価格が大きいことが分かる。

このため１が「新品、未使用」で５が「全体的に悪い」だと推測できる。

In [44]:
del train_df, test_df
gc.collect()
train_df = pd.read_csv(path + 'train.tsv', sep='\t', encoding='utf-8', nrows = 1000)
test_df = pd.read_csv(path+'test.tsv', sep='\t', encoding='utf-8', nrows = 1000)

プロトタイプ実装のためここからはデータ量を削減して作業を行った。

### 前処理の実装と説明

今回前処理として下記を行った。

- 欠損値を埋める
- 商品説明欄の'No description yet'は空欄の場合挿入されるワードなのでこれは空白文字に置き換える

### 特徴量エンジニアリング

In [45]:
# 欠損値埋める
train_df['item_condition_id'].fillna(2, inplace=True)
test_df['item_condition_id'].fillna(2, inplace=True)
train_df['shipping'].fillna(0, inplace=True)
test_df['shipping'].fillna(0, inplace=True)

# 説明欄の補正
train_df['item_description'].fillna('', inplace=True)
train_df['item_description'] = train_df['item_description'].replace('No description yet', '')
test_df['item_description'].fillna('', inplace=True)
test_df['item_description'] = test_df['item_description'].replace('No description yet', '')

### FS1 説明欄の３文字以上の単語数をカウントする　num_words_item_description

In [46]:
# 商品説明欄の単語数をカウントする
def create_count_features(df_data):
    def lg(text):
        text = [x for x in text.split() if len(x) >= 3]
        return len(text)

    df_data['num_words_item_description'] = df_data['item_description'].apply(lg).astype(np.uint16)
    return df_data
train_df = create_count_features(train_df)
test_df = create_count_features(test_df)

### FS2 メインカテゴリーのカウントベクトル

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
def concat_train_test(train,test):
# trainとtestのidカラム名を変更する
    train = train.rename(columns = {'train_id':'id'})
    test = test.rename(columns = {'test_id':'id'})

    # 両方のセットへ「is_train」のカラムを追加
    # 1 = trainのデータ、0 = testデータ
    train['is_train'] = 1
    test['is_train'] = 0

    # trainのprice(価格）以外のデータをtestと連結
    train_test_combine = pd.concat([train.drop(['price'], axis=1),test],axis=0)
    return train_test_combine

def split_cat(text):
    # textを'/'で区切る
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")


def item_des(df):
    df['general_cat'], df['subcat_1'], df['subcat_2'] = \
    zip(*df['category_name'].apply(lambda x: split_cat(x)))
    return df

def to_categorical(df):
    df['general_cat'] = df['general_cat'].astype('category')
    df['subcat_1'] = df['subcat_1'].astype('category')
    df['subcat_2'] = df['subcat_2'].astype('category')
    df['item_condition_id'] = df['item_condition_id'].astype('category')
    return df


def CV(df, length):
    
    cv = CountVectorizer(min_df=5)
    cv.fit(df['general_cat'])
    X_train_category1 = cv.transform(df['general_cat'][:length])
    X_test_category1 = cv.transform(df['general_cat'][length:])

    return X_train_category1,X_test_category1

def get_main_category(train,test):
    len_train = len(train)
    concat_df = concat_train_test(train, test)
    concat_df = item_des(concat_df)
    concat_df = to_categorical(concat_df)
    train_cat1, test_cat1 = CV(concat_df, len_train)

    return train_cat1, test_cat1

train_cat1, test_cat1 = get_main_category(train_df, test_df)

### FS3 商品説明欄のカウントベクトル化

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer


def concat_train_test(train,test):
# trainとtestのidカラム名を変更する
    train = train.rename(columns = {'train_id':'id'})
    test = test.rename(columns = {'test_id':'id'})

    # 両方のセットへ「is_train」のカラムを追加
    # 1 = trainのデータ、0 = testデータ
    train['is_train'] = 1
    test['is_train'] = 0

    # trainのprice(価格）以外のデータをtestと連結
    train_test_combine = pd.concat([train.drop(['price'], axis=1),test],axis=0)
    return train_test_combine

def item_des_tfidf(df,length):
    tv = TfidfVectorizer(max_features=1000,
                             ngram_range=(1, 3),
                             stop_words='english')
    tv.fit(df['item_description'])

    X_description = tv.transform(df['item_description'])
    X_train_description = tv.transform(df['item_description'][:length])
    X_test_description = tv.transform(df['item_description'][length:])

    return X_train_description,X_test_description

def get_X_description(train,test):
    len_train = len(train)
    concat_df = concat_train_test(train, test)
    train_description, test_description = item_des_tfidf(concat_df, len_train)

    return train_description, test_description

train_item_description, test_item_description = get_X_description(train_df, test_df)

In [49]:
from scipy.sparse import csr_matrix, hstack, vstack

use_cols = ['item_condition_id', 'shipping', 'num_words_item_description']
X_train = train_df[use_cols]
y_train = train_df['price'].values
X_test = test_df[use_cols]
if PRODOCTION == False:
    y_test = test_df['price'].values

# sparse_matrix化 カウントベクトル系の特徴量を追加
X_train = hstack((X_train, train_cat1, train_item_description)).tocsr()
X_test = hstack((X_test, test_cat1, test_item_description)).tocsr()

### [注意]ここまででX_train,X_testは特徴量エンジニアリングした項目と合わせてsparse_matrixにしておく。

## 評価関数の作成

In [50]:
from sklearn.metrics import mean_squared_log_error 
from sklearn.metrics import make_scorer

# 評価関数を定義
def root_mean_squared_log_error(truth, pred):
    error = np.log1p(truth) - np.log1p(pred)
    return np.sqrt(np.sum(error*error)/ len(truth) )

rmsle = make_scorer(root_mean_squared_log_error, greater_is_better=False)

## 1.LinearModel リッジ回帰での学習

In [51]:
###################
# リッジ回帰で学習
###################
from sklearn.linear_model import Ridge

model_ridge = Ridge(alpha=20)
model_ridge.fit(X_train, y_train)

y_pred_ridge = model_ridge.predict(X_test)

### 交差検証

例としてリッジ回帰のみの交差検証。

In [52]:
from sklearn.model_selection import KFold


kf = KFold(n_splits=3)
for train_index, test_index in kf.split(X_train):
    cv_X_train, cv_X_test = X_train[train_index], X_train[test_index]
    cv_y_train, cv_y_test = y_train[train_index], y_train[test_index]

    model_ridge = Ridge(alpha=20)
    model_ridge.fit(cv_X_train, cv_y_train)
    print(root_mean_squared_log_error(cv_y_test, model_ridge.predict(cv_X_test)))

0.8464497611479546
0.833194791143947
0.7840237702365039


## 2.GBM:XGBoostモデルでの学習

In [25]:
###################
# XGBoostで学習させてみる
###################
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
# 動かすパラメータを明示的に表示
params = {
    "learning_rate":[0.05,0.1],
    "max_depth": [2,3,5],
    "subsample":[0.5,0.8,0.9,1],
    "colsample_bytree": [0.5,1.0],
}

params = {
                # Feiyang: 10. 把 7 改成了 8
                'num_leaves': 2 ** 8 - 1,
                'objective': 'regression_l2',
                # Feiyang: 11. 把 8 改成了 9
                'max_depth': 9,
                'min_data_in_leaf': 50,
                # Feiyang: 12. 把 0.01 改成了 0.007 并同时改了下面的 Num_round 和 early_stopping_rounds
                'learning_rate': 0.007,
                'feature_fraction': 0.6,
                # Feiyang: 13. 把 0.75 改成了 0.8
                'bagging_fraction': 0.8,
                'bagging_freq': 1,
                'metric': 'rmse',
                'num_threads': 4,
                'seed': 2018,
            }

# モデルにインスタンス生成
model = xgb.XGBRegressor()
# ハイパーパラメータ探索
cv = GridSearchCV(model, params, cv = 3, scoring=rmsle) 
cv.fit(X_train, y_train)

if PRODOCTION==False:
    print(root_mean_squared_log_error(y_test, cv.predict(X_test)))

y_pred_xgb = cv.predict(X_test)

## 3.アンサンブル（予測値の平均）

一番簡単な方法で予測値の合計をモデル数で割る。
コードは下記の通りだが、今回は採用せず、４のスタッキングで最終的な予測値を出力する。

In [27]:
y_pred_array = np.c_[y_pred_ridge, y_pred_xgb]
y_pred_ensemble = np.average(y_pred_array, axis=1)

## 4. スタッキング（XGBoost）

検証は下記のように行う。

1. 訓練データをn分割し、n-1が訓練データ、1がテストデータとする。
2. それぞれの分割に対して、各モデルで学習、予測を行う。
3. 各モデルが予測した出力を説明変数としてXGBoostで学習する。


> [参考](http://segafreder.hatenablog.com/entry/2016/05/24/235822)

In [76]:
from sklearn.model_selection import KFold

######################
# STEP1 : 2モデルでの学習
######################

score_ridge_list = []
score_xgb_list = []

predict = np.zeros((y_train.shape[0], 2))

kf = KFold(n_splits=3)
for train_index, test_index in kf.split(X_train):
    cv_X_train, cv_X_test = X_train[train_index], X_train[test_index]
    cv_y_train, cv_y_test = y_train[train_index], y_train[test_index]

    # Ridge回帰
    model_ridge = Ridge(alpha=20)
    model_ridge.fit(cv_X_train, cv_y_train)
    y_predict_ridge = model_ridge.predict(cv_X_test)

    # xgboostモデルの作成
    reg = xgb.XGBRegressor()

    # ハイパーパラメータ探索
    reg_cv = GridSearchCV(reg, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)
    reg_cv.fit(cv_X_train, cv_y_train)
    print(reg_cv.best_params_, reg_cv.best_score_)

    # 改めて最適パラメータで学習
    model_xgb = xgb.XGBRegressor(**reg_cv.best_params_)
    model_xgb.fit(cv_X_train, cv_y_train)
    y_predict_xgb = model_xgb.predict(cv_X_test)
    
    score_ridge = root_mean_squared_log_error(cv_y_test, y_predict_ridge)
    score_xgb = root_mean_squared_log_error(cv_y_test, y_predict_xgb)
    
    score_ridge_list.append(score_ridge)
    score_xgb_list.append(score_xgb)
    
    print("Ridge回帰：{}".format(score_ridge))
    print("XGBoost：{}".format(score_xgb))
    
    predict[test_index, 0] = y_predict_ridge
    predict[test_index, 1]= y_predict_xgb    

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    6.4s finished


{'max_depth': 2, 'n_estimators': 50} -0.07441824488856552
Ridge回帰：0.8464474511314641
XGBoost：0.8869277746238913
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    6.2s finished


{'max_depth': 2, 'n_estimators': 50} -0.1833287294720582
Ridge回帰：0.8332148417389875
XGBoost：0.8709576674468583
Fitting 3 folds for each of 9 candidates, totalling 27 fits
{'max_depth': 2, 'n_estimators': 50} -0.2612983780989679
Ridge回帰：0.7840364494834698
XGBoost：0.8148636252475677


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    6.2s finished


In [75]:
######################
# STEP2 : 予測値を説明変数とした学習でスコアを検証
######################

score_stacking_list = []

kf = KFold(n_splits=3)
for train_index, test_index in kf.split(predict):
    cv_X_train, cv_X_test = predict[train_index, :], predict[test_index, :]
    cv_y_train, cv_y_test = y_train[train_index], y_train[test_index]
    
    # xgboostモデルの作成
    reg = xgb.XGBRegressor()

    # ハイパーパラメータ探索
    reg_cv = GridSearchCV(reg, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)
    reg_cv.fit(X_train, y_train)
    print(reg_cv.best_params_, reg_cv.best_score_)

    # 改めて最適パラメータで学習
    model_xgb = xgb.XGBRegressor(**reg_cv.best_params_)
    model_xgb.fit(cv_X_train, cv_y_train)
    y_predict_stacking = model_xgb.predict(cv_X_test)
    
    score = root_mean_squared_log_error(cv_y_test, y_predict_stacking)
    print("スタッキング：{}".format(score))
    score_stacking_list.append(score)


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x1a40432198>>
Traceback (most recent call last):
  File "/Users/kzfm/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/xgboost/core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    8.1s finished


{'max_depth': 2, 'n_estimators': 50} -0.12414173972858455
スタッキング：0.8390638344996908
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    7.5s finished


{'max_depth': 2, 'n_estimators': 50} -0.12414173972858455
スタッキング：1.0960539597494117
Fitting 3 folds for each of 9 candidates, totalling 27 fits
{'max_depth': 2, 'n_estimators': 50} -0.12414173972858455
スタッキング：0.7813018558939326


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    8.0s finished


In [92]:
######################
# STEP3 : スコアを確認する
######################

print("RIDGE      score:", np.average(score_ridge_list))
print("XGB         score:", np.average(score_xgb_list))
print("Staking   score:", np.average(score_stacking_list))

RIDGE      score: 0.8212329141179738
XGB         score: 0.8575830224394392
Staking   score: 0.905473216714345


よければ全訓練データで学習してテストデータを予測する。

1000件データでスコアが上昇していないが、スコアが上がったとして進める。

In [87]:
######################
# スタッキングで学習
######################

# Ridge回帰
model_ridge = Ridge(alpha=20)
model_ridge.fit(X_train, y_train)
y_predict_ridge = model_ridge.predict(X_test)

# xgboostモデルの作成
reg = xgb.XGBRegressor()

# ハイパーパラメータ探索
# reg_cv = GridSearchCV(reg, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)
# reg_cv.fit(cv_X_train, cv_y_train)
# print(reg_cv.best_params_, reg_cv.best_score_)

# XGBoostモデルで学習
model_xgb = xgb.XGBRegressor(**reg_cv.best_params_)
model_xgb.fit(X_train, y_train)
y_predict_xgb = model_xgb.predict(X_test)

# テストデータの予測値を説明変数とする
y_predict_two_modle = np.array([y_predict_ridge, y_predict_xgb]).T

model_xgb = xgb.XGBRegressor(**reg_cv.best_params_)
model_xgb.fit(predict, y_train)
y_predict_stacking = model_xgb.predict(y_predict_two_modle)

## 予測値を提出ファイルに出力する

In [89]:
mysubmission=pd.DataFrame()
mysubmission['test_id']=test_df['test_id']
mysubmission['price'] = y_predict_stacking
mysubmission.to_csv('ensemble.csv', index=False)

## パイプライン化

前処理部分、特徴量エンジニアリング部分をモジュール化した。

メインファイル　sprint8_main.py

mercariディレクトリに下記の４ファイルを配置します。

- 説明欄の単語数（３文字以上）カウント：create_count_feature.py
- メインカテゴリーのカウントベクトル化：get_main_category.py
- 説明欄のカウントベクトル化：item_des_feature.py
- ブランド名のダミー変数：create_count_feature.py

## メモリ管理

今回のようにsparse matrixを使ったり、大規模なデータを扱う際にはメモリ使用量に気をつけながら開発を行う必要がある。

**memory_profiler**を導入すると現在のメモリ使用量や変数がどれくらいメモリを使っているかが分かる。

pip install memory_profiler


### 1. astype('np.uint16')などで型を小さくする。

pd.read_csvは数値型のデータはint64型を確保してしまう。
そのため型や数値の最小、最大値を確認して適切な型を指定するとメモリの使用を抑えられる。

In [102]:
sample_df = pd.read_csv(path+'test.tsv', sep='\t', encoding='utf-8' , nrows=10)

In [104]:
sample_df.dtypes

test_id               int64
name                 object
item_condition_id     int64
category_name        object
brand_name           object
shipping              int64
item_description     object
dtype: object

In [105]:
sample_df.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


今回はshippingに着目する。

In [106]:
sample_df['shipping'].max()

1

In [108]:
sample_df['shipping'].min()

0

In [109]:
sample_df['shipping'] = sample_df['shipping'].astype('uint8')
sample_df.dtypes

test_id               int64
name                 object
item_condition_id     int64
category_name        object
brand_name           object
shipping              uint8
item_description     object
dtype: object

型が変更された。このように数値型のデータは贅沢にメモリを使うので注意する。

### 2. read_csvでdtypeを指定　

データ型が予め分かっている場合はread_csv時に型を指定することができる。
データの内容を確認した後はこの方法でメモリを削減できる。

In [114]:
sample_df = pd.read_csv(path+'test.tsv', sep='\t', encoding='utf-8' , nrows=10, dtype={'item_condition_id': 'uint8', 'shipping': 'uint8'})
sample_df.dtypes

test_id               int64
name                 object
item_condition_id     uint8
category_name        object
brand_name           object
shipping              uint8
item_description     object
dtype: object

### 3. read_csvでchunksize指定して指定データ数でループさせる

データ数を制限して実行できる。

In [94]:
for df_submission in pd.read_csv(path+'test.tsv', sep='\t', encoding='utf-8' , chunksize=80000):
    print(df_submission.shape)

(80000, 7)
(80000, 7)
(80000, 7)
(80000, 7)
(80000, 7)
(80000, 7)
(80000, 7)
(80000, 7)
(53359, 7)


### 4. ガベージコレクションで使用し終わった変数を削除

In [96]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [97]:
memit 

peak memory: 703.46 MiB, increment: 0.18 MiB


In [98]:
memit train_df = pd.read_csv(path + 'train.tsv', sep='\t', encoding='utf-8', nrows = 500000)

peak memory: 868.23 MiB, increment: 164.77 MiB


１５０MBほど使用量が増えていることが分かる。

変数を削除して、ガベージコレクションを使うとメモリを開放してくれる。

In [100]:
import gc
del train_df
gc.collect

<function gc.collect>

In [101]:
memit

peak memory: 723.33 MiB, increment: 0.00 MiB


メモリ使用量が元に戻った。

メモリ節約の方法はいくつかあるのでデータ量が多そうな場合は計測してどうするか考える。

## マルチプロセッシング

Grobal Interpreter Lock(GIL)
1つのインタプリタ上で複数スレッドを同時実行されないようにして、スレッドセーフを木にする必要がなくなり通常の動作が高速になる。

しかしこのままでは複数のCPUを積んでいていも同時に実行できるスレッドは常に一つに制限されてしまいハードを活かすことはできません。

この制限を回避するために、標準ライブラリに**multiprocessing**というマルチプロセスを扱うライブラリがあります。

> パーフェクトpython P35参照

今回はテスト関数（ある項目値を２倍にする）の処理をマルチコアで行うような処理を記述してみる。

In [122]:
from multiprocessing import Pool


cores=2
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, cores)
    pool = Pool(cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [130]:
def test_func(df):
    print(df['test_id'].head())
    print(df.shape)
    print('')
    df['item_condition_id'] = df['item_condition_id']*2 
    return df

In [131]:
sample_df = pd.read_csv(path+'test.tsv', sep='\t', encoding='utf-8' , nrows=1000)

result_df = parallelize_dataframe(sample_df, test_func)

0    0
1    1
2    2
3    3
4    4
Name: test_id, dtype: int64500    500
501    501
502    502
503    503
504    504
Name: test_id, dtype: int64

(500, 7)
(500, 7)




In [132]:
result_df.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",2,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",2,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,2,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,4,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,6,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


このようにコアにプロセスを割り振ってデータを集約することで複数コアを余すことなく使用することができる。